In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re  
import json
import pprint
import xlwt
import xlrd
import openpyxl
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
workbook = openpyxl.Workbook()  #创建一个Excel文件,用于存储输出数据
worksheet = workbook.create_sheet('text') #创建一个sheet
row=1
rbook = xlrd.open_workbook(r'C:\Users\dengx\Desktop\2-8856.xlsx')    #####数据文件名
# sheets方法返回对象列表,[<xlrd.sheet.Sheet object at 0x103f147f0>]
rbook.sheets()
# xls默认有3个工作簿,Sheet1,Sheet2,Sheet3
rsheet = rbook.sheet_by_index(0)  # 取第一个工作簿
# 循环工作簿的所有行
curl="https://m.ctrip.com/restapi/soa2/16593/json/getHouseCommentList"
for i in rsheet.get_rows():
    print(i)
    unitID = int(i[0].value)
    if i[28].value!='none':
        pageSize=int(i[28].value)
    else:
        pageSize=0
    payloadData={
        "unitID":unitID,"pageIndex":0,"pageSize":pageSize,"tagParameter":"1,0","head":{"cid":"09031029212120301011","ctok":"","cver":"1.0","lang":"01","sid":"8888","syscode":"09","auth":"","xsid":"","extension":[{"name":"page","value":"10650016818"},{"name":"pagename","value":"detail"},{"name":"terminaltype","value":"100"},{"name":"apiv","value":"1"},{"name":"traceid","value":"93009641-eaf0-2754-9188-211c000594bf"},{"name":"channelid","value":"211"},{"name":"inclient","value":"true"},{"name":"allianceSid","value":"135371"},{"name":"allianceId","value":"4899"},{"name":"awakeUnion","value":"{\"OUID\":\"\",\"AllianceID\":\"4899\",\"SID\":\"135371\",\"SourceID\":\"\"}"},{"name":"userAgent","value":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36"}]}
    }
    payloadHeader={
        "content-type": "application/json",
        "origin": "https://inn.ctrip.com"
    }
    print(unitID,pageSize)
    cresponse=rq.post(curl,data=json.dumps(payloadData),headers=payloadHeader)
    # 评论长度，对应pagesize
    data=json.loads(cresponse.text)  #json就是字典数据
    #pprint.pprint(data)
    comments=data['data']['comments']
    for comment in comments:
        worksheet.cell(row,1).value = comment['unitID']       #房源ID
        comment['commentDetail']= ILLEGAL_CHARACTERS_RE.sub(r'', comment['commentDetail'])
        worksheet.cell(row,2).value = comment['commentDetail']  #评论内容
        worksheet.cell(row,3).value = comment['createTime']     #评论创建时间
        comment['hotelName']= ILLEGAL_CHARACTERS_RE.sub(r'', comment['hotelName'])
        worksheet.cell(row,4).value = comment['hotelName']    #名称
        worksheet.cell(row,5).value = comment['overall']        #总分
        comment['replyDetail']= ILLEGAL_CHARACTERS_RE.sub(r'', comment['replyDetail'])
        worksheet.cell(row,6).value = comment['replyDetail']    #回复内容
        worksheet.cell(row,7).value = comment['replySubmitTime'] #回复时间
        comment['scoreTitle']= ILLEGAL_CHARACTERS_RE.sub(r'', comment['scoreTitle'])
        worksheet.cell(row,8).value = comment['scoreTitle'] 
        row=row+1
workbook.save('C:\\Users\\dengx\\Desktop\\xiecheng2-8856.xlsx')         ###结果保存地址